In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("Experiment").getOrCreate()

In [4]:
spark

In [5]:
data_path = "C:\\Users\\Pratiush\\Desktop\\PratLib\\PratLib\\data\\sales_data.csv"

# Reading a CSV file into a DataFrame
df = spark.read.csv(data_path, header=True, inferSchema=True)

In [6]:
df.show()

+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|User_ID|Product_ID|Gender|  Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|
+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|1000001| P00069042|     F| 0-17|        10|            A|                         2|             0|                 3|              NULL|              NULL|    8370|
|1000001| P00248942|     F| 0-17|        10|            A|                         2|             0|                 1|                 6|                14|   15200|
|1000001| P00087842|     F| 0-17|        10|            A|                         2|             0|                12|              NULL|              NULL|    1422

In [105]:
from pyspark.ml.feature import OneHotEncoder as SparkOneHotEncoder
from pyspark.sql.functions import when
import warnings

In [116]:
class OneHotEncoder:
    def __init__(self, **kwargs):
        self.encoder = None
        self.indexer = None
        self.indexer_output_col = None
        self.encoder_output_col = None

    def fit(self, df, input_col):
        self.indexer_output_col = f"{input_col}_index"
        self.encoder_output_col = f"{input_col}_encoded"
        
        self.indexer = StringIndexer(inputCol=input_col, outputCol=self.indexer_output_col).fit(df)
        indexed_df = self.indexer.transform(df)
        
        self.encoder = SparkOneHotEncoder(inputCol=self.indexer_output_col, outputCol=self.encoder_output_col).fit(indexed_df)
        return self

    def transform(self, df):
        if not self.encoder or not self.indexer:
            raise AttributeError("The encoder has not been fitted yet.")
        
        indexed_df = self.indexer.transform(df)
        new_df = self.encoder.transform(indexed_df)
        new_df = new_df.drop(self.indexer_output_col)
        return new_df

class LabelEncoder:
    def __init__(self, **kwargs):
        self.indexer = StringIndexer(**kwargs)

    def fit(self, df, input_col):
        self.indexer.setInputCol(input_col).setOutputCol(f"{input_col}_new")
        self.model = self.indexer.fit(df)
        return self
            
    def transform(self, df):
        return self.model.transform(df)

class map:
    def __init__(self):
        pass
    def mapper(self, df, column_name, dic):
        for key, value in dic.items():
            df = df.withColumn(
                column_name,
                when(df[column_name] == key, value).otherwise(df[column_name])
            )
        return df


In [117]:
ohe = OneHotEncoder()
ohe.fit(df,"Age")

In [118]:
# indexed_df.select('Age_new').distinct().collect()

In [119]:
ohe.transform(df)

DataFrame[User_ID: int, Product_ID: string, Gender: string, Age: string, Occupation: int, City_Category: string, Stay_In_Current_City_Years: string, Marital_Status: int, Product_Category_1: int, Product_Category_2: int, Product_Category_3: int, Purchase: int, Age_encoded: vector]

In [120]:
le = LabelEncoder()

In [121]:
le.fit(df,"Age")

In [122]:
le.transform(df)

DataFrame[User_ID: int, Product_ID: string, Gender: string, Age: string, Occupation: int, City_Category: string, Stay_In_Current_City_Years: string, Marital_Status: int, Product_Category_1: int, Product_Category_2: int, Product_Category_3: int, Purchase: int, Age_new: double]

In [123]:
map = Mapper()

In [124]:
df = Mapper.map(df, "Age", {'0-17':0, '18-25':1,"26-35":2, "36-45":3,"46-50":4,"51-55":5,"55+":6})

TypeError: Mapper.map() missing 1 required positional argument: 'dic'

In [98]:
df.select("Age").distinct().collect()

[Row(Age='3'),
 Row(Age='0'),
 Row(Age='5'),
 Row(Age='6'),
 Row(Age='1'),
 Row(Age='4'),
 Row(Age='2')]

In [13]:
df.columns

['User_ID',
 'Product_ID',
 'Gender',
 'Age',
 'Occupation',
 'City_Category',
 'Stay_In_Current_City_Years',
 'Marital_Status',
 'Product_Category_1',
 'Product_Category_2',
 'Product_Category_3',
 'Purchase',
 'Gender_new']